In [ ]:
# OPTION 1 - BACKWARD STRATEGY (BS)
def backward_strategy_prune(model, target_layer_index):
    """
    Removes a specific hidden layer and reconfigures weights to correctly connect
    adjacent layers. Redirects weights from the pruned layer to the next layer,
    adjusting their size if necessary.
    """
    
    layers = model.layers

    if target_layer_index < 2 or target_layer_index >= len(layers) - 1:
        raise ValueError("The hidden layer index must be between 2 and the total number of hidden layers.")

    print("\n=== STARTING PRUNING PROCESS ===")
    print(f"Target layer index to remove: {target_layer_index}")
    print(f"Total layers in model: {len(layers)}")

    # Involved layers
    previous_layer = layers[target_layer_index - 1]
    layer_to_prune = layers[target_layer_index]
    next_layer = layers[target_layer_index + 1]

    print("\n--- Layers Involved ---")
    print(f"Previous Layer: {previous_layer.name}")
    print(f"Layer to Prune: {layer_to_prune.name}")
    print(f"Next Layer: {next_layer.name}")

    print(f"\nRetrieving weights and biases from target layer: {layer_to_prune.name}")
    print(f"Using '{previous_layer.name}' as reference layer.")

    # Retrieve weights and biases
    input_weights, input_bias = layer_to_prune.get_weights()
    print(f"\nTarget layer weights (shape {input_weights.shape}):\n{input_weights}")
    print(f"Target layer bias:\n{input_bias}")

    ref_weights, ref_bias = previous_layer.get_weights()
    print(f"\nReference layer weights (shape {ref_weights.shape}):\n{ref_weights}")
    print(f"Reference layer bias:\n{ref_bias}")

    num_neurons_prev = input_weights.shape[0]
    num_neurons_prune = input_weights.shape[1]
    num_neurons_next = next_layer.units

    print("\n--- Weight Adjustment ---")
    print(f"Neurons in previous layer: {num_neurons_prev}")
    print(f"Neurons in layer to prune: {num_neurons_prune}")
    print(f"Neurons in next layer: {num_neurons_next}")

    if input_weights.shape[1] > num_neurons_next:
        print("\n------------------------- OPTION 1, BS ----------------------")
        print("\nReducing weights and bias: selecting neurons based on absolute mean similarity.")

        # 1. Target layer means
        origin_means = np.mean(input_weights, axis=0)
        print("\nMean vector (target layer):")
        print(np.round(origin_means, 4))

        # 2. Next layer means
        next_weights, _ = next_layer.get_weights()
        reference_means = np.mean(next_weights, axis=0)
        print("\nMean vector (next layer):")
        print(np.round(reference_means, 4))

        # 3. Select indices of the closest neurons (without repetition)
        selected_indices = []
        used_indices = set()
        for ref_val in reference_means:
            differences = np.abs(origin_means - ref_val)
            sorted_order = np.argsort(differences)
            for idx in sorted_order:
                if idx not in used_indices:
                    selected_indices.append(idx)
                    used_indices.add(idx)
                    break

        print("\nFinal selected indices based on mean similarity:")
        print(selected_indices)

        # 4. Filter weights and biases using selected indices
        input_weights = input_weights[:, selected_indices]
        input_bias = input_bias[selected_indices]

    elif input_weights.shape[1] < num_neurons_next:
        print("\n------- Next layer neuron count is larger; BS not applicable --------")
        return model

    print(f"New weights shape for connecting previous to next layer: {input_weights.shape}")
    print(f"New adjusted bias shape: {input_bias.shape}")

    new_weights = input_weights
    new_bias = input_bias

    # FIXED FOR Keras 3/TF 2.16.1
    # 1. Get model input shape
    full_input_shape = model.input_shape
    
    # 2. Handle multiple inputs (if list)
    if isinstance(full_input_shape, list):
        full_input_shape = full_input_shape[0]
        
    # 3. Extract features dimension (e.g., 12 from (None, 12))
    if isinstance(full_input_shape, tuple):
        input_dim = full_input_shape[-1] 
    else:
        raise ValueError(f"Could not determine model input shape: {full_input_shape}")

    print(f"\n--- Creating new model with input_shape={input_dim} ---")

    inputs = keras.Input(shape=(input_dim,))
    x = inputs

    # Iterate through original model layers to build the new one
    for i, layer in enumerate(layers[1:]): 
        # Skip non-Dense layers
        if not isinstance(layer, keras.layers.Dense):
            continue 
            
        # Skip the pruned layer
        if i + 1 == target_layer_index:
            print(f"Skipping layer: {layer.name}")
            continue
            
        # Reconstruct layer with original attributes
        x = keras.layers.Dense(layer.units, activation=layer.activation, name=layer.name)(x)
        print(f"Adding layer: {layer.name} with {layer.units} units")

    new_model = keras.Model(inputs=inputs, outputs=x)

    print("\n--- Adjusting weights in the new model ---")
    for layer in new_model.layers:
        if isinstance(layer, keras.layers.Dense):
            # 1. Assign the merged weights to the "next" layer
            if layer.name == next_layer.name:
                layer.set_weights([new_weights, new_bias])
                print(f"✅ Assigned adjusted weights to layer: {layer.name}")
            # 2. Copy original weights for all other layers
            else:
                original_layer = next((c for c in model.layers if c.name == layer.name), None)
                if original_layer is not None:
                    layer.set_weights(original_layer.get_weights())
                    print(f"✅ Copied original weights to layer: {layer.name}")

    # Re-compile model with original configuration
    optimizer = keras.optimizers.Adam(epsilon=1e-8, learning_rate=learning_rate)
    new_model.compile(optimizer=optimizer, loss='mae', metrics=['mse', 'mae'])

    print("\n=== PROCESS COMPLETE ===")
    new_model.summary()

    return new_model

In [ ]:
# OPTION 2 - FORWARD STRATEGY (FS)
def forward_strategy_prune(model, target_layer_index):
    import keras
    import numpy as np
    
    layers = model.layers

    if target_layer_index < 1 or target_layer_index >= len(layers) - 1:
        raise ValueError("Index must correspond to a hidden layer (not input or output).")

    print("\n=== STARTING PRUNING PROCESS (FS) ===")

    previous_layer = layers[target_layer_index - 1]
    layer_to_prune = layers[target_layer_index]
    next_layer = layers[target_layer_index + 1]

    print("\n--- Layers Involved ---")
    print(f"Previous Layer: {previous_layer.name}")
    print(f"Layer to Prune: {layer_to_prune.name}")
    print(f"Next Layer: {next_layer.name}")

    # Original weights from the next layer
    next_weights, next_bias = next_layer.get_weights()

    print(f"\nLayer to prune: {layer_to_prune.name}")
    print(f"Next layer: {next_layer.name}")
    print(f"Original next layer weights shape: {next_weights.shape}")

    print(f"\nRetrieving weights and bias from target layer: {layer_to_prune.name}")
    print(f"Using next layer '{next_layer.name}' as reference.")

    input_weights, input_bias = layer_to_prune.get_weights()
    print(f"\nTarget layer weights (shape {input_weights.shape}):\n{input_weights}")
    print(f"Target layer bias:\n{input_bias}")

    ref_weights, ref_bias = next_layer.get_weights()
    print(f"\nReference layer weights (shape {ref_weights.shape}):\n{ref_weights}")
    print(f"Reference layer bias:\n{ref_bias}")

    num_neurons_prev = input_weights.shape[0]
    num_neurons_prune = input_weights.shape[1]
    num_neurons_next = next_layer.units

    print("\n--- Weight Adjustment ---")
    print(f"Neurons in previous layer: {num_neurons_prev}")
    print(f"Neurons in layer to prune: {num_neurons_prune}")
    print(f"Neurons in next layer: {num_neurons_next}")

    # --- CORRECTION 1: Determine new input dimension ---
    if hasattr(previous_layer, "units"):
        # If the previous layer is Dense, use its units
        new_input_dim = previous_layer.units
    else:
        # If previous is InputLayer, use model's input shape
        full_input_shape = model.input_shape
        if isinstance(full_input_shape, list):
            full_input_shape = full_input_shape[0]
        # Extract feature dimension (e.g., 12 from (None, 12))
        new_input_dim = full_input_shape[-1] 
    
    output_neurons = next_layer.units

    print(f"\nAdjusting weights for layer '{next_layer.name}' from {next_weights.shape} → ({new_input_dim}, {output_neurons})")

    if next_weights.shape[0] > new_input_dim:
        print("\n🔻 Row reduction via mean similarity")
        original_means = np.mean(next_weights, axis=1)
        # Generate reference means with uniform spacing
        reference_means = np.linspace(np.min(original_means), np.max(original_means), new_input_dim)

        print("\n▶ Original row means:")
        print(original_means)

        print("\n▶ Generated reference means (uniform spacing):")
        print(reference_means)

        selected_indices = []
        used_indices = set()
        for i, ref in enumerate(reference_means):
            distances = np.abs(original_means - ref)
            sorted_order = np.argsort(distances)
            print(f"\n→ Comparing with reference {i} ({ref:.4f}):")
            print("  Distances:", distances)
            print("  Order:", sorted_order)

            for idx in sorted_order:
                if idx not in used_indices:
                    selected_indices.append(idx)
                    used_indices.add(idx)
                    print(f"  ✅ Selected index {idx} (mean {original_means[idx]:.4f})")
                    break

        print("\n✅ Final indices selected to keep:")
        print(selected_indices)

        # Slice the weights using the selected row indices
        next_weights = next_weights[selected_indices, :]

    elif next_weights.shape[0] < new_input_dim:
        print("\n----------- Next layer has more neurons; FS not applicable -----------")
        return model
        
    print(f"\n✅ Final adjusted weights shape: {next_weights.shape}")

    # --- CORRECTION 2: Get input shape for model reconstruction ---
    full_input_shape = model.input_shape
    if isinstance(full_input_shape, list):
        full_input_shape = full_input_shape[0]
    input_dim = full_input_shape[-1] 
    
    print(f"\n--- Creating new model with input_shape={input_dim} ---")

    inputs = keras.Input(shape=(input_dim,))
    x = inputs

    for i, layer in enumerate(layers[1:]):
        if i + 1 == target_layer_index:
            print(f"Skipping pruned layer: {layer.name}")
            continue
        
        # Ensure only Dense layers are processed
        if not isinstance(layer, keras.layers.Dense):
            continue
            
        x = keras.layers.Dense(layer.units, activation=layer.activation, name=layer.name)(x)

    new_model = keras.Model(inputs=inputs, outputs=x)

    # Assign weights to the new model
    for layer in new_model.layers:
        if isinstance(layer, keras.layers.Dense):
            if layer.name == next_layer.name:
                layer.set_weights([next_weights, next_bias])
                print(f"✅ Assigned new weights to layer: {layer.name}")
            else:
                # Copy original weights for unchanged layers
                original_layer = next((c for c in model.layers if c.name == layer.name), None)
                if original_layer:
                    layer.set_weights(original_layer.get_weights())
                    print(f"✅ Copied original weights to layer: {layer.name}")

    # Recompile with the global learning rate
    optimizer = keras.optimizers.Adam(epsilon=1e-8, learning_rate=learning_rate)
    new_model.compile(optimizer=optimizer, loss='mae', metrics=['mse', 'mae'])

    print("\n=== PROCESS COMPLETE ===")
    new_model.summary()

    return new_model

In [ ]:
# STANDARD PRUNING
def standard_pruning(
    original_model: keras.Model,
    layer_index_to_remove: int,
) -> keras.Model:
    """
    Standard pruning that removes a hidden layer and reconstructs the model.
    Note: This method does not preserve weights for the adjacent layers; 
    it simply defines a new architecture with one less layer.
    """

    # --- 1. Extract configuration and index ---

    # Hidden layers in your model start at index 1 (index 0 is InputLayer)
    hidden_dense_layers = original_model.layers[1:-1]
    
    # The provided hidden layer index is 1-based (1, 2, 3...)
    # For the `hidden_dense_layers` list, we use 0-based indexing, so we subtract 1.
    remove_idx = layer_index_to_remove - 1

    if remove_idx < 0 or remove_idx >= len(hidden_dense_layers):
        print(f"ERROR: Index {layer_index_to_remove} is out of the hidden layer range (1 to {len(hidden_dense_layers)}).")
        return original_model

    # --- 2. Define New Architecture (Layer Pruning) ---

    # Get the unit count for all original hidden layers
    original_hidden_units = [layer.units for layer in hidden_dense_layers]
    
    # Create the new unit list by omitting the selected layer
    reduced_hidden_units = (
        original_hidden_units[:remove_idx] + 
        original_hidden_units[remove_idx+1:]
    )

    print(f"Pruning hidden layer #{layer_index_to_remove} with {original_hidden_units[remove_idx]} units.")
    print(f"New architecture (hidden units): {reduced_hidden_units}")

    # --- 3. Model Reconstruction ---

    # Extract input shape and the original output layer activation
    input_shape = original_model.input_shape[1:]
    output_activation = original_model.layers[-1].activation.__name__
    
    input_layer = keras.Input(shape=input_shape)
    x = input_layer

    # Add reduced hidden layers
    for units in reduced_hidden_units:
        # Assuming 'tanh' as the standard activation for hidden layers
        x = keras.layers.Dense(units, activation='tanh')(x)

    # Add output layer
    # Maintains 24 units and the original output activation
    output_layer = keras.layers.Dense(24, activation=output_activation)(x)

    # Create the reduced functional model
    reduced_model = keras.Model(inputs=input_layer, outputs=output_layer)

    # Compile the reduced model
    # Using the global 'learning_rate' variable defined previously
    optimizer = keras.optimizers.Adam(epsilon=1e-8, learning_rate=learning_rate)
    reduced_model.compile(optimizer=optimizer, loss='mae', metrics=['mae', 'mse'])
    
    print("\n--- Reduced Model Summary ---")
    reduced_model.summary()
    print("----------------------------------\n")
    
    return reduced_model

In [ ]:
#FORMA 2 - FS
def forwardStrategyPrune(model, indice_capa_oculta):    
    capas = model.layers

    if indice_capa_oculta < 1 or indice_capa_oculta >= len(capas) - 1:
        raise ValueError("El índice debe ser el de una capa oculta, no input ni output.")

    print("\n=== INICIO DEL PROCESO ===")

    capa_antes = capas[indice_capa_oculta - 1]
    capa_eliminar = capas[indice_capa_oculta]
    capa_despues = capas[indice_capa_oculta + 1]

    print("\n--- Capas involucradas ---")
    print(f"Capa antes: {capa_antes.name}")
    print(f"Capa a eliminar: {capa_eliminar.name}")
    print(f"Capa después: {capa_despues.name}")

    # Pesos originales de la capa siguiente
    pesos_siguiente, bias_siguiente = capa_despues.get_weights()

    print(f"\nCapa a eliminar: {capa_eliminar.name}")
    print(f"Capa siguiente: {capa_despues.name}")
    print(f"Forma original de pesos de la capa siguiente: {pesos_siguiente.shape}")

    print(f"\nObteniendo pesos y bias de la capa a eliminar: {capa_eliminar.name}")
    print(f"Usando la capa posterior '{capa_despues.name}' como referencia.")

    pesos_entrada, bias_entrada = capa_eliminar.get_weights()
    print(f"\nPesos de la capa a eliminar (forma {pesos_entrada.shape}):\n{pesos_entrada}")
    print(f"Bias de la capa a eliminar:\n{bias_entrada}")

    pesos_ref, bias_ref = capa_despues.get_weights()
    print(f"\nPesos de la capa de referencia (forma {pesos_ref.shape}):\n{pesos_ref}")
    print(f"Bias de la capa de referencia:\n{bias_ref}")

    num_neuronas_antes = pesos_entrada.shape[0]
    num_neuronas_eliminar = pesos_entrada.shape[1]
    num_neuronas_siguiente = capa_despues.units

    print("\n--- Ajuste de pesos ---")
    print(f"Número de neuronas de la capa previa: {num_neuronas_antes}")
    print(f"Número de neuronas de la capa a eliminar: {num_neuronas_eliminar}")
    print(f"Número de neuronas de la capa siguiente: {num_neuronas_siguiente}")

    # --- CORRECCIÓN 1: Obtener la nueva dimensión de entrada ---
    if hasattr(capa_antes, "units"):
        # Si la capa anterior es Dense, usamos sus units
        neuronas_entrada_nueva = capa_antes.units
    else:
        # Si la capa anterior es la InputLayer, la dimensión es el input_shape del modelo
        input_shape_full = model.input_shape
        if isinstance(input_shape_full, list):
            input_shape_full = input_shape_full[0]
        # Extraer la dimensión de la característica (ej: 12 de (None, 12))
        neuronas_entrada_nueva = input_shape_full[-1] 
    # --- FIN CORRECCIÓN 1 ---
    
    neuronas_salida = capa_despues.units

    print(f"\nAjustando pesos de la capa '{capa_despues.name}' de forma ({pesos_siguiente.shape}) → ({neuronas_entrada_nueva}, {neuronas_salida})")

    if pesos_siguiente.shape[0] > neuronas_entrada_nueva:
        print("\n🔻 Reducción de filas por similitud de medias")
        medias_originales = np.mean(pesos_siguiente, axis=1)
        medias_referencia = np.linspace(np.min(medias_originales), np.max(medias_originales), neuronas_entrada_nueva)

        print("\n▶ Medias de filas originales:")
        print(medias_originales)

        print("\n▶ Medias de referencia generadas (espaciado uniforme):")
        print(medias_referencia)

        indices_seleccionados = []
        usadas = set()
        for i, ref in enumerate(medias_referencia):
            distancias = np.abs(medias_originales - ref)
            orden = np.argsort(distancias)
            print(f"\n→ Comparando con referencia {i} ({ref:.4f}):")
            print("  Distancias:", distancias)
            print("  Orden:", orden)

            for idx in orden:
                if idx not in usadas:
                    indices_seleccionados.append(idx)
                    usadas.add(idx)
                    print(f"  ✅ Seleccionado índice {idx} (media {medias_originales[idx]:.4f})")
                    break

        print("\n✅ Índices seleccionados para mantener:")
        print(indices_seleccionados)

        pesos_siguiente = pesos_siguiente[indices_seleccionados, :]

    elif pesos_siguiente.shape[0] < neuronas_entrada_nueva:
        print("\n-----------La siguiente capa tiene mas neuronas que la actual, FS no apta-----------")
        return model
        
    print(f"\n✅ Forma final de pesos ajustados: {pesos_siguiente.shape}")

    # --- CORRECCIÓN 2: Obtener la dimensión de entrada para recrear el modelo ---
    input_shape_full = model.input_shape
    if isinstance(input_shape_full, list):
        input_shape_full = input_shape_full[0]
    # Extraer la dimensión de la característica
    input_shape = input_shape_full[-1] 
    # --- FIN CORRECCIÓN 2 ---
    
    print(f"\n--- Creando nuevo modelo con input_shape={input_shape} ---")

    inputs = keras.Input(shape=(input_shape,))
    x = inputs

    for i, capa in enumerate(capas[1:]):
        if i + 1 == indice_capa_oculta:
            print(f"Omitiendo capa eliminada: {capa.name}")
            continue
        
        # Asegurarse de que solo agregamos capas Dense
        if not isinstance(capa, keras.layers.Dense):
            continue
            
        x = keras.layers.Dense(capa.units, activation=capa.activation, name=capa.name)(x)

    nuevo_modelo = keras.Model(inputs=inputs, outputs=x)

    # Asignar pesos
    for capa in nuevo_modelo.layers:
        if isinstance(capa, keras.layers.Dense):
            if capa.name == capa_despues.name:
                capa.set_weights([pesos_siguiente, bias_siguiente])
                print(f"✅ Asignados nuevos pesos a la capa: {capa.name}")
            else:
                capa_original = next((c for c in model.layers if c.name == capa.name), None)
                if capa_original:
                    capa.set_weights(capa_original.get_weights())
                    print(f"✅ Copiados pesos originales a la capa: {capa.name}")

    optimizer = keras.optimizers.Adam(epsilon=1E-8, learning_rate=lr)
    nuevo_modelo.compile(optimizer=optimizer, loss='mae', metrics=['mse', 'mae'])

    print("\n=== FINAL DEL PROCESO ===")
    nuevo_modelo.summary()

    return nuevo_modelo

In [ ]:
def inspect_model_weights(model):
    """
    Inspects the weights and biases of each layer in a Keras model.

    Parameter:
    - model: Keras/TensorFlow model object.

    Prints:
    - Name, type of each layer, and its weights if they are trainable.
    """
    for i, layer in enumerate(model.layers):
        # Retrieve layer weights and biases
        weights = layer.get_weights()  
        print(f"\n🔹 Layer {i}: {layer.name} ({layer.__class__.__name__})")

        if weights:  # Check if the layer contains trainable parameters
            for j, parameter_block in enumerate(weights):
                print(f"  - Weight Block {j}: {parameter_block.shape}\n{parameter_block}")
        else:
            print("  - ⚠️ This layer does not have trainable weights.")


In [ ]:
# Strategy Selection: 1 = Backward, 2 = Forward, 3 = Standard
pruning_strategy = 2  
target_hidden_layer = 4

if pruning_strategy == 1:
    # Option 1: Backward Strategy Pruning (BS)
    model_pruned = backward_strategy_prune(model, target_hidden_layer)
elif pruning_strategy == 2:
    # Option 2: Forward Strategy Pruning (FS)
    model_pruned = forward_strategy_prune(model, target_hidden_layer)
elif pruning_strategy == 3:
    # Option 3: Standard Layer Pruning (Classic)
    model_pruned = standard_pruning(model, target_hidden_layer)


=== INICIO DEL PROCESO ===

--- Capas involucradas ---
Capa antes: dense_2
Capa a eliminar: dense_3
Capa después: dense_4

Capa a eliminar: dense_3
Capa siguiente: dense_4
Forma original de pesos de la capa siguiente: (60, 60)

Obteniendo pesos y bias de la capa a eliminar: dense_3
Usando la capa posterior 'dense_4' como referencia.

Pesos de la capa a eliminar (forma (90, 60)):
[[-0.08882892  0.06672331  0.1106843  ... -0.20323947  0.06148278
   0.2421428 ]
 [ 0.19140379 -0.21269378 -0.09361155 ...  0.22304621  0.17197263
   0.004413  ]
 [ 0.06765034  0.22428113 -0.2594401  ...  0.08092441 -0.23108414
   0.04495906]
 ...
 [ 0.07980245  0.17283972 -0.14182174 ... -0.18219991 -0.06776182
   0.2398633 ]
 [ 0.1251675  -0.20480551 -0.07700592 ... -0.10676853 -0.02044641
  -0.15120286]
 [ 0.04675413  0.06360561  0.07661088 ...  0.12218941  0.00556134
   0.13549331]]
Bias de la capa a eliminar:
[ 2.73897741e-02 -1.60246436e-02  8.16922449e-03  1.95053068e-03
  4.56092320e-02 -3.51369567e-02

In [ ]:
# Inspect the weights of the model before the pruning process
inspect_model_weights(model)


🔹 Capa 0: input_layer_14 (InputLayer)
  - ⚠️ Esta capa no tiene pesos entrenables.

🔹 Capa 1: dense_65 (Dense)
  - Pesos 0: (12, 40)
[[-0.09370826  0.1648086  -0.04056354  0.20126562 -0.01161632 -0.08252753
  -0.29845887 -0.20267463  0.04238677  0.04885807 -0.3260325   0.16161369
   0.22421041  0.19623907  0.03051327 -0.03902812  0.2757984   0.04351014
   0.04160324  0.1769534  -0.259557    0.27386674 -0.13647202  0.24432117
   0.22435176  0.13586353 -0.04853562 -0.20521729  0.12013845 -0.06374133
  -0.02477405  0.02528322 -0.01590031  0.04751011 -0.17829372 -0.22820728
   0.2971012  -0.2446874  -0.05363576 -0.03390842]
 [-0.29390913  0.09263787 -0.33330426 -0.18944603 -0.25264004  0.06865249
   0.26102686 -0.18672867 -0.07013597  0.09645228  0.06126654 -0.10778011
  -0.04747451 -0.22576983  0.1012185   0.02811892 -0.17445968  0.27496547
   0.05527949  0.08184452 -0.17765856  0.11106235 -0.16169366  0.00613702
  -0.3017288   0.14146754  0.33003038  0.2176185   0.01011087 -0.06358144
 

In [ ]:
# Inspect the weights of the model after the pruning process
inspect_model_weights(model_pruned)


🔹 Capa 0: input_layer_5 (InputLayer)
  - ⚠️ Esta capa no tiene pesos entrenables.

🔹 Capa 1: dense_66 (Dense)
  - Pesos 0: (12, 60)
[[-0.20643388  0.01737381 -0.00188578  0.00556127 -0.10054328 -0.2519836
  -0.1667882   0.07135277 -0.17053965  0.1275013   0.04329761  0.08041992
  -0.1192439  -0.11776188 -0.01921184  0.10459366 -0.11548372  0.08950839
   0.1624882  -0.16331919  0.03551297  0.11267059 -0.04133165  0.03294617
   0.13955274  0.12717809 -0.11313774 -0.1067669   0.06422283  0.09431536
  -0.12213927 -0.08069505 -0.21390247 -0.07917381 -0.16724993 -0.14692762
  -0.12473761  0.15342925  0.22386019 -0.08436671 -0.13789415  0.17510635
  -0.14657678  0.07016288 -0.0576023  -0.18092096 -0.12540042 -0.05903937
  -0.21648926  0.10846126 -0.04665549 -0.17362091 -0.19607621  0.17490526
  -0.14085928  0.14255056 -0.01416865 -0.10074482 -0.04997594 -0.01044213]
 [ 0.20612204 -0.06837474  0.16418466  0.11965851 -0.05126453  0.16008167
  -0.22026812 -0.1146555  -0.12831193  0.21519008 -0.

In [ ]:
# FOR LOOP CODE TO ITERATE OVER LAYERS TO BE PRUNED USING 3 METHODOLOGIES

# Indices are 1-based
layers_to_prune = [1, 2, 3, 4, 5]
pruning_strategy = 3  # Fix the methodology
strategy_map = {1: "BS", 2: "FS", 3: "Standard"}
strategy_text = strategy_map.get(pruning_strategy, "Unknown Option")

comparison_results = []

print(f"--- Starting evaluation of {strategy_text} Pruning on {len(layers_to_prune)} layers ---")
print("-" * 60)

# --- Load dataset for denormalization ---
original_data = pd.read_csv('dataset.csv')
min_val = original_data['consumption'].min()
max_val = original_data['consumption'].max()

# --- Experimentation Loop ---

for target_layer in layers_to_prune:
    # Configure Early Stopping for each iteration
    early_stopping_pruned = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True
    )
    
    print(f"\n[EXPERIMENT] Pruning hidden layer number: {target_layer} using {strategy_text}")
    
    # Select the pruning strategy
    if pruning_strategy == 1:
        model_pruned = backward_strategy_prune(model, target_layer)
    elif pruning_strategy == 2:
        model_pruned = forward_strategy_prune(model, target_layer)
    elif pruning_strategy == 3:
        model_pruned = standard_pruning(model, target_layer)

    # Fine-tuning the pruned model
    start_time_ft = time.time()
    history_pruned = model_pruned.fit(
        X_train_norm, 
        y_train_norm, 
        epochs=epochs, 
        batch_size=batch_size, 
        validation_data=(X_val_norm, y_val_norm), 
        callbacks=[early_stopping_pruned]
    )
    end_time_ft = time.time()
    
    # Perform predictions on test set
    y_pred_pruned = model_pruned.predict(X_test_norm)

    # --- Denormalization Process (N x 24 Arrays) ---
    y_pred_denorm = y_pred_pruned * (max_val - min_val) + min_val
    y_test_denorm = y_test * (max_val - min_val) + min_val

    # Calculate global MAPE
    mape_pruned = mean_absolute_percentage_error(y_test_denorm, y_pred_denorm) * 100

    # --- MAPE STANDARD DEVIATION CALCULATION ---

    # 1. Flatten arrays for point-by-point error calculation
    y_true_flat = y_test_denorm.flatten()
    y_pred_flat = y_pred_denorm.flatten()

    # 2. Handle division by zero (using epsilon)
    epsilon = np.finfo(np.float32).eps 
    y_true_for_mape = np.copy(y_true_flat)
    y_true_for_mape[y_true_for_mape == 0] = epsilon

    # 3. Calculate Absolute Percentage Error (APE)
    absolute_percentage_errors = (np.abs(y_true_flat - y_pred_flat) / y_true_for_mape) * 100

    # 4. Calculate Standard Deviation
    std_dev_mape = np.std(absolute_percentage_errors) 

    # --- Print Iteration Results ---
    print("-" * 35)
    print(f'Methodology: {strategy_text} | Target Layer: {target_layer}')
    print(f'Mean Absolute Percentage Error (MAPE): {mape_pruned:.4f}%')
    print(f"MAPE Standard Deviation: {std_dev_mape:.3f}%")
    
    ft_duration = end_time_ft - start_time_ft
    print(f"Total fine-tuning time: {ft_duration:.2f} seconds")
    
    final_epochs = len(history_pruned.history['loss'])
    print(f"Epochs used in fine-tuning: {final_epochs}")
    
    # Store results for final summary
    comparison_results.append({
        'strategy': strategy_text,
        'pruned_layer': target_layer,
        'std_dev': std_dev_mape,
        'mape': mape_pruned,
        'duration': ft_duration,
        'epochs': final_epochs
    })
    print("-" * 60)

# --- Final Summary Table ---
print("\n\n=============== FINAL PRUNING SUMMARY BY LAYER ===============")
print(f"{'Layer':<10} | {'MAPE':<12} | {'Std Dev':<12} | {'Time (s)':<10} | {'Epochs':<8}")
print("-" * 65)
for res in comparison_results:
    print(f"#{res['pruned_layer']:<9} | {res['mape']:<11.4f}% | {res['std_dev']:<11.4f}% | {res['duration']:<9.2f} | {res['epochs']:<7}")
print("================================================================")

--- Iniciando evaluación de BS Pruning en 4 capas ---
------------------------------------------------------------

[EXPERIMENTO] Poda de la capa oculta número: 2 usando BS

=== INICIO DEL PROCESO ===
Índice de la capa a eliminar: 2
Total de capas en el modelo: 7

--- Capas involucradas ---
Capa antes: dense_31
Capa a eliminar: dense_32
Capa después: dense_33

Obteniendo pesos y bias de la capa a eliminar: dense_32
Usando la capa previa 'dense_31' como referencia.

Pesos de la capa a eliminar (forma (60, 60)):
[[-0.19268203  0.1084566  -0.21906939 ... -0.01030684  0.08241396
  -0.04914217]
 [-0.29224822 -0.15678838  0.01726513 ... -0.06435319  0.07672251
  -0.30379033]
 [ 0.1924373  -0.02836844 -0.12709993 ... -0.019751   -0.5414242
  -0.12879966]
 ...
 [-0.35584357  0.1123801   0.41310415 ... -0.2304633  -0.40038466
   0.18349741]
 [-1.0776225   0.03957691  0.22652973 ... -0.12808564 -0.85976946
   0.716171  ]
 [-0.06882188 -0.34597257 -0.6551341  ...  0.4174504  -0.6751328
   0.36498

Model: "functional_19"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_19 (InputLayer)     │ (None, 168)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 60)             │        10,140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_35 (Dense)                │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 24)             │         1,464 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,584 (88.22 KB)

 Trainable params: 22,584 (88.22 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_22 (InputLayer)     │ (None, 168)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 60)             │        10,140 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 60)             │         3,660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_36 (Dense)                │ (None, 24)             │         1,464 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,584 (88.22 KB)

 Trainable params: 22,584 (88.22 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.0412 - mae: 0.0412 - mse: 0.0149 - val_loss: 0.0183 - val_mae: 0.0183 - val_mse: 7.0018e-04
Epoch 2/300
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.0175 - mae: 0.0175 - mse: 6.1885e-04 - val_loss: 0.0165 - val_mae: 0.0165 - val_mse: 5.5244e-04
Epoch 3/300
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0162 - mae: 0.0162 - mse: 5.2469e-04 - val_loss: 0.0160 - val_mae: 0.0160 - val_mse: 5.0812e-04
Epoch 4/300
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0155 - mae: 0.0155 - mse: 4.7951e-04 - val_loss: 0.0151 - val_mae: 0.0151 - val_mse: 4.6170e-04
Epoch 5/300
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0149 - mae: 0.0149 - mse: 4.4741e-04 - val_loss: 0.0177 - val_mae: 0.0177 - val_mse: 5.5600e-04
Epoch 6/300
1906/1906 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.0146 - mae: 0.0146 - mse: 4.2756e-04 - val_loss: 0.0144 - val_mae: 0.0144 - val_mse: 4.1952e-04
Epoch 7/300
1906/1906 ━━━━━━━━━━━━━━━━